<a href="https://colab.research.google.com/github/Wiqou/Cheatsheets/blob/main/Tensorflow_migration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2019-09-30부로 TensorFlow 2.0
Eager Execution<br>
v1의 경우 그래프 생성과 실행 분리->세션을 열고 그래프를 실행하는 시점에서 계산되는 구조<br>
성능을 위한 선택이었지만 반직관적<br>
v2에서는 바로 그래프의 특정값을 계산할수 있는 Eager Execution이 기본<br>
따라서 모든 코드는 쓰여진 라인 순서에 따라 실행<br>
1. Local한 처리가 가능해짐<br>
v1의 경우 암시적으로 global인 형태로 처리<br>
예를 들어 tf.Variable을 선언하면 global 그래프에 선언한 tf.Variable가 추가<br>
tf.global_variables_initializer() API를 호출시 모든 선언된 tf.Variable들에 초기값을 할당<br>
2. tf.placeholder&tf.Session 삭제와 @tf.function<br>
tf.Session API 삭제 -> @tf.function이용(기본연산 취급)<br>
tf.placeholder API 삭제 -> 지정된 함수에 값을 입력<br>
outputs = session.run(f(placeholder), feed_dict={placeholder: input}) #v1<br>
outputs = f(input) #  v2

In [ ]:
in_a = tf.placeholder(dtype=tf.float32, shape=(2)) #v1
in_b = tf.placeholder(dtype=tf.float32, shape=(2)) #1.global하니까 input설계필수
def forward(x):
  with tf.variable_scope("matmul"): #1.matmul이랑 같은 거 쓰게 with로 잠깐 불러오기
    W = tf.get_variable("W", initializer=tf.ones(shape=(2,2)), regularizer=tf.contrib.layers.l2_regularizer(0.04)) #1.local하게 처리하려면 함수안쪽에 정의
    b = tf.get_variable("b", initializer=tf.zeros(shape=(2)))
    return W * x + b 
out_a = forward(in_a)
out_b = forward(in_b)
reg_loss = tf.losses.get_regularization_loss(scope="matmul")
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer()) #2.내장함수도 적용
  outs = sess.run([out_a, out_b, reg_loss], feed_dict={in_a: [1, 0], in_b: [0, 1]}) #2.이때 입력 및 계산

In [ ]:
W = tf.Variable(tf.ones(shape=(2,2)), name="W") #v2
b = tf.Variable(tf.zeros(shape=(2)), name="b") #get_variable필요없고 바로입력 #어차피 local
@tf.function
def forward(x):
     return W * x + b
out_a = forward([1,0]) #함수에 바로 입력
print(out_a) #친숙하게